In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Quante-Näherungs-Optimierungsalgorithmus

*Zitschätzig: 22 Minute uf emene Heron-r3-Prozässor (HINWYS: Das isch nur e Schätzig. Dini Uusfüehrigszit cha abwiiche.)*
## Hintergrund
Dä Tutorial zeigt, wie mer de **Quante-Näherungs-Optimierungsalgorithmus (QAOA)** — e hybride (quante-klassischi) iterativi Methode — im Rahme vo Qiskit-Patterns umsetzt. Du lösch zerschte s'**Maximum-Cut**- (oder **Max-Cut**-) Problem für e chline Graph und lernsch de, wie mer's in Utility-Scale uusfüehre cha. Alli Hardware-Uusfüehrige im Tutorial sötte innerhalb vom Zitlimit vom frei zugängliche Open Plan laafe.

S'Max-Cut-Problem isch es Optimierungsproblem, wo schwer z'löse isch (genauer gseit, isch's es *NP-schweres* Problem) mit verschiedene Aawändigsfäll im Clustering, i de Netzwärchwiisseschaft und i dr statistische Physik. Dä Tutorial betrachtet ene Graph vo Knöte, wo dur Kante verbunde sind, und wott d'Knöte in zwee Mengene ufteile, sodass d'Aaazahl vo de dur dä Schnitt duerquärte Kante maximiert wird.

![Illustratio vom Max-Cut-Problem](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Voraussetzige
Bevor du mit däm Tutorial afangsch, vergewisseri dich, dass du s'Folgende installiert hesch:
- Qiskit SDK v1.0 oder neuer, mit Unterstitzig für [Visualisieriig](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 oder neuer (`pip install qiskit-ibm-runtime`)

Zudem bruuchsch du Zuegriff uf e Instanz uf de [IBM Quantum Platform](/guides/cloud-setup). Bitte beachte, dass dä Tutorial nöd im Open Plan uusgfüehrt wärde cha, will er Workloads mit [Sessions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session) betribt, wo nur im Premium Plan verfüegbar sind.
## Setup

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Teil I. QAOA im chline Massstab
Im erschte Teil vo däm Tutorial wird e chlinmassstäblichs Max-Cut-Problem bruucht, zum d'Schritte zum Löse vomeme Optimierungsproblem mit emene Quantecomputer z'illustriere.

Um etwas Kontext z'gäh, bevor mer das Problem uf ene Quantealgorithmus abbilde, cha mer s'Max-Cut-Problem besser als klassischs kombinatorischs Optimierungsproblem verstah, indem mer zerschte d'Minimierig vomere Funktion $f(x)$ betrachtet

$$
\min_{x\in {0, 1}^n}f(x),
$$

wo dr Iiput $x$ en Vektor isch, desse Komponente de Knöte vom Graph entspreche. De Beschränkt jedi vo dene Komponente uf $0$ oder $1$ (was bedütet, dass si im Schnitt drinn oder nöd drinn sind). Dises chline Bispiil nimmt ene Graph mit $n=5$ Knöte.

Du chasch e Funktion vomeme Paar vo Knöte $i,j$ schriibe, wo azeigt, ob d'entsprechende Kante $(i,j)$ im Schnitt isch. Zum Bispiil isch d'Funktion $x_i + x_j - 2 x_i x_j$ nur denn 1, wenn entweder $x_i$ oder $x_j$ gleich 1 isch (was bedütet, dass d'Kante im Schnitt isch), und sonsch 0. S'Problem vom Maximiere vo de Kante im Schnitt cha so formuliert wärde:

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

was als Minimierig umgschriibe wärde cha:

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

S'Minimum vo $f(x)$ isch i däm Fall denn, wenn d'Aaazahl vo de dur de Schnitt duerquärte Kante maximal isch. Wie du gsesch, het das no nüüt mit Quantecomputing z'tüe. Du muesch das Problem in öppis umformuliere, wo e Quantecomputer verstah cha.
Initialisier dis Problem, indem du ene Graph mit $n=5$ Knöte erstellsch.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Schritt 1: Klassischi Iiput uf es Quanteproblem abbilden
Dr erschte Schritt vom Pattern isch s'Abbilde vom klassische Problem (Graph) uf Quante**schaltkreis** und **Operatore**. Dafür git's drei Hauptschritte:

1. E Reihe vo mathematische Umformulierige nutze, zum das Problem in dr Notation vo Quadratic Unconstrained Binary Optimization (QUBO) darzstelle.
2. S'Optimierungsproblem als Hamiltonian umschriibe, desse Grundzuestand dr Lösig entspricht, wo d'Kostenfunktion minimiert.
3. Ene Quanteschaltkreis erstelle, wo de Grundzuestand vo däm Hamiltonian dur en ähnliche Prozäss wie Quanteannealing vorbereitet.

**Hinwys:** Bim QAOA-Verfahre witt du schlussändlich ene Operator (**Hamiltonian**) ha, wo d'**Kostenfunktion** vo uusem hybride Algorithmus repräsentiert, sowie ene parametrisierte Schaltkreis (**Ansatz**), wo Quantezueständ mit Kandidatenlösige für s'Problem darstellt. Du chasch von dene Kandidatenzueständ samplen und si de mit dr Kostenfunktion bewärte.

#### Graph &rarr; Optimierungsproblem
Dr erschte Schritt bim Abbilde isch e Notationsänderig. S'Folgende drückt das Problem in QUBO-Notation uus:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

wo $Q$ e $n\times n$-Matrix vo reelle Zahle isch, $n$ dr Aaazahl vo Knöte im Graph entspricht, $x$ dr Vektor vo binäre Variabele isch, wie oben iiigfüehrt, und $x^T$ de Transposita vom Vektor $x$ bedütet.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Optimierungsproblem &rarr; Hamiltonian
Du chasch s'QUBO-Problem de als **Hamiltonian** umformuliere (hie e Matrix, wo d'Energie vomeme System repräsentiert):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Umformulierigs-Schritte vom QAOA-Problem zum Hamiltonian**
</summary>

Um z'zeige, wie das QAOA-Problem so umgschriibe wärde cha, ersetzt mer zerschte d'binäre Variabele $x_i$ dur e neui Satz vo Variabele $z_i\in{-1, 1}$ via

$$
x_i = \frac{1-z_i}{2}.
$$

Hie gsiehsch, dass wenn $x_i$ gleich $0$ isch, $z_i$ gleich $1$ sy mueß. Wenn d'$x_i$'s dur d'$z_i$'s im Optimierungsproblem ($x^TQx$) ersetzt wärde, chan mer e äquivalenti Formulierig erhalte.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Wenn mer jetzt $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ definiert, de Vorfaktor weglaat und de konstante $n^2$-Term entfernt, chöme mer zu de zwee äquivalente Formulierige vom gliche Optimierungsproblem.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Hie hängt $b$ vo $Q$ ab. Beachte, dass zum Erhalte vo $z^TQz + b^Tz$ dr Faktor 1/4 und e konstante Verschiebig vo $n^2$ weggloo worde sind, wo bi dr Optimierig kei Rolle spieled.

Um jetzt e Quanteformulierig vom Problem z'erhalte, wärde d'$z_i$-Variabele zu Pauli-$Z$-Matrize befördert, also zu $2\times 2$-Matrize vo dr Form

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Wenn du die Matrize im Optimierungsproblem oben iisetzt, erhälsch de folgende Hamiltonian

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Vergiss ou nöd, dass d'$Z$-Matrize im rechnerische Raum vom Quantecomputer iibettet sind, also im Hilbert-Raum dr Grösse $2^n\times 2^n$. Drum söttsch Ausdrück wie $Z_iZ_j$ als Tensorprodukt $Z_i\otimes Z_j$ iibettet im $2^n\times 2^n$-Hilbert-Raum verstah. Zum Bispiil isch dr Term $Z_1Z_3$ bi emem Problem mit füüf Entscheidigsvariabele als $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ z'verstah, wo $I$ d'$2\times 2$-Identitätsmatrix isch.*
</details>

Dä Hamiltonian wird dr **Kostenfunktions-Hamiltonian** gnennt. Er het d'Eigenschaft, dass sy Grundzuestand dr Lösig entspricht, wo d'**Kostenfunktion $f(x)$ minimiert**.
Drum muesch jetzt, zum dis Optimierungsproblem z'löse, de Grundzuestand vo $H_C$ (oder en Zuestand mit grossem Üüberlappig damit) uf em Quantecomputer vorbereite. S'Samplen von däm Zuestand wird de mit hoher Wahrschynlichkeit d'Lösig für $\min~f(x)$ ergäh.
Mir betrachte jetzt de Hamiltonian $H_C$ für s'**Max-Cut**-Problem. Me ordnet jedem Knote vom Graph en Qubit im Zuestand $|0\rangle$ oder $|1\rangle$ zue, wo dr Wärt azeigt, in welcher Mengene dr Knote isch. S'Ziel vom Problem isch, d'Aaazahl vo Kante $(v_1, v_2)$ z'maximiere, für wo $v_1 = |0\rangle$ und $v_2 = |1\rangle$ gilt, oder umgekehrt. Wenn mer dem $Z$-Operator jedem Qubit zuordnet, wo gilt

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

de ghört e Kante $(v_1, v_2)$ zum Schnitt, wenn dr Eigenwärt vo $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$ isch; mit andere Wort, d'Qubits vo $v_1$ und $v_2$ sind verschidde. Entsprechend ghört $(v_1, v_2)$ nöd zum Schnitt, wenn dr Eigenwärt vo $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$ isch. Beachte, dass es nöd uf de exakte Qubit-Zuestand vu jedem Knote aakommt, sondern nur ob si ueber e Kante gliche oder verschidde sind. S'Max-Cut-Problem fordert e Zueordig vo de Qubits uf de Knöte, wo de Eigenwärt vom folgende Hamiltonian minimiert

$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

Mit andere Wort, $b_i = 0$ für alli $i$ bim Max-Cut-Problem. Dr Wärt $Q_{ij}$ bezeichnet s'Gwicht vo dr Kante. I däm Tutorial betrachte mer ene ungewichtete Graph, also $Q_{ij} = 1.0$ für alli $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Schritt 2: Problem für d'Quantehardware-Uusfüehrig optimiere
Dr Schaltkreis oben enthält e Reihe vo Abstraktione, wo nütz sind zum Dänke über Quantealgorithme, sich aber nöd direkt uf dr Hardware uusfüehre lönd. Um uf emene QPU laufe z'chönne, mueß dr Schaltkreis e Reihe vo Operatione durchlaufe, wo de **Transpilierigs**- oder **Schaltkreisoptimierigs**-Schritt vo em Pattern usmache.

D'Qiskit-Bibliothek bietet e Reihe vo **Transpilierungspassene**, wo e groossi Bandbreite vo Schaltkreistransformatione abdecke. Du muesch sicherstelle, dass din Schaltkreis für din Zwäck **optimiert** isch.

D'Transpilierig cha mehere Schritte umfasse, zum Bispiil:

* **Erscht-Abbildig** vo de Qubits im Schaltkreis (wie Entscheidigsvariabele) uf physikalischi Qubits uf em Gerät.
* **Abrole** vo de Iischtruktiöne im Quanteschaltkreis zu de Hardware-nativen Iischtruktiöne, wo de Backend versteht.
* **Routing** vo allne Qubits im Schaltkreis, wo mitanand interagiere, zu physikalische Qubits, wo benachbart sind.
* **Fählerunderdückig** dur s'Addiere vo Einzel-Qubit-Gates, zum Rüsche mit dynamischem Entkopplen z'underdrücke.

Meh Informatiöne über d'Transpilierig finsch in uuserer [Dokumentatio](/guides/transpile).

Dr folgende Code transformiert und optimiert de abstrakten Schaltkreis in es Format, wo für d'Uusfüehrig uf einere vo de über d'Cloud zugängliche Devices mit em **Qiskit IBM Runtime Service** bereit isch.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Schritt 3: Uusfüehre mit Qiskit-Primitiven
Bim QAOA-Workflow wärde d'optimale QAOA-Parameter in ememe iterative Optimierigsloup gfunde, wo e Reihe vo Schaltkreisauswärtige dureführt und ene klassische Optimierer bruucht, um d'optimale $\beta_k$- und $\gamma_k$-Parameter z'finde. Dä Uusfüehrigsloup wird über die Schritte uusgfüehrt:

1. D'Anfangsparameter definiere
2. E neui `Session` instanziere, wo de Optimierigsloup und s'Primitiv enthält, das zum Samplen vom Schaltkreis bruucht wird
3. Sobald e optimali Satz vo Parametere gfunde worden isch, de Schaltkreis e letzts Mal uusfüehre, um e schlussendlichs Verteilung z'erhalte, wo im Nachbearbeitigsschritt bruucht wird.
#### Schaltkreis mit Anfangsparameter definiere
Mir fange mit willkürlich gwählte Parametere aa.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Backend und Uusfüehrigsprimitiv definiere
Bruuch d'**Qiskit-Runtime-Primitive**, um mit IBM&reg;-Backends z'interagiere. D'zwee Primitive sind Sampler und Estimator, und d'Wahl vom Primitiv hängt davon ab, was für Mässige du uf em Quantecomputer uusfüehre witt. Für d'Minimierig vo $H_C$ bruuchsch de Estimator, will d'Mässig vo dr Kostenfunktion eifach dr Erwärtigswärt $\langle H_C \rangle$ isch.
#### Uusfüehre
D'Primitive biete e Vielfalt vo [Uusfüehrigsmodi](/guides/execution-modes), zum Workloads uf Quantedevices z'plane, und en QAOA-Workflow lauft iterativ in einere Session.

![Illustratio, wo s'Verhalte vo einzelne Job, Batch und Session Runtime-Modi zeigt.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

Du chasch d'sampler-basierti Kostenfunktion in de SciPy-Minimierungsroutine iisetze, zum d'optimale Parameter z'finde.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Sobald du d'optimale Parameter für de Schaltkreis gfunde hesch, chasch dene Parameter zuewyyse und d'schlussendlichi Verteilung samplen, wo mit de optimierte Parametere ermittlet wird. Hie sollsch s'*Sampler*-Primitiv bruuche, will's d'Wahrschynlichkeitsverteilig vo Bitstrig-Mässige isch, wo de optimale Schnitt vom Graph entspreche.

**Hinwys:** Das bedütet, ene Quantezuestand $\psi$ im Rächner vorzubereite und in denn z'mässe. E Mässig kollabiert de Zuestand in ene einzelne Berechnigsbasis-Zuestand — zum Bispiil `010101110000...` — was einere Kandidatenlösig $x$ für uuses ursprünglichs Optimierungsproblem ($\max f(x)$ oder $\min f(x)$ je nach Ufgab) entspricht.

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Schritt 4: Nachbearbeitig und Ergebnis im gwünschte klassische Format
Dr Nachbearbeitigsschritt interpretiert de Sampling-Uusgab, um e Lösig für dis ursprünglichs Problem z'liiefere. I däm Fall interessierst di für de Bitstrig mit dr höchste Wahrschynlichkeit, will däm de optimale Schnitt bestimmt. D'Symmetrie im Problem erlaubt vier möglichs Lösige, und dr Sampling-Prozäss wird eine davon mit en chli höherer Wahrschynlichkeit zurückgäh, aber du gsiehsch in dr ufgzeichnete Verteilig unge, dass vier Bitstrige deutlich wahrschynlicher sind als d'übrige.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Beste Schnitt visualisiere
Vom optimale Bitstrig uus chasch de dä Schnitt uf em ursprüngliche Graph visualisiere.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

Und de Wärt vom Schnitt bärechne:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Teil II. Hochskaliere!
Du hesch Zuegriff uf viili Devices mit über 100 Qubits uf dr IBM Quantum&reg; Platform. Wähl eines uus, uf wäm du Max-Cut uf emene gewichtete Graph mit 100 Knöte lösch. Das isch es «Utility-Scale»-Problem. D'Schritte zum Uufbau vom Workflow sind dieselbe wie oben, aber mit emem vill grösserem Graph.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Schritt 1: Klassischi Iiput uf es Quanteproblem abbilden
#### Graph &rarr; Hamiltonian
Konvertier zerschte de Graph, wo du löse witt, direkt in ene Hamiltonian, wo für QAOA geignet isch.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamiltonian &rarr; Quanteschaltkreis

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Uusgab vo de vorherige Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Schritt 2: Problem für d'Quanteuusfüehrig optimiere
Um de Schaltkreisoptimierigs-Schritt uf Utility-Scale-Probleme z'skaliere, chasch d'Hochleistigungs-Transpilierungsstrategien nutze, wo im Qiskit SDK v1.0 iiigfüehrt worde sind. Ander Wärchzüüg umfassend de neue Transpiler-Service mit [KI-gstiitzte Transpilierungspassene](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Sobald d'optimale Parameter vom Uusfüehre vom QAOA uf em Gerät gfunde worde sind, wärde die dem Schaltkreis zuewyyse.